In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, BatchNormalization
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
df_train = pd.read_csv("D:/COEP/Project/Main/Final Review/Dataset/mitbih_train.csv", header=None)

In [4]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87549,0.807018,0.494737,0.536842,0.529825,0.491228,0.484211,0.456140,0.396491,0.284211,0.136842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87550,0.718333,0.605000,0.486667,0.361667,0.231667,0.120000,0.051667,0.001667,0.000000,0.013333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87551,0.906122,0.624490,0.595918,0.575510,0.530612,0.481633,0.444898,0.387755,0.322449,0.191837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87552,0.858228,0.645570,0.845570,0.248101,0.167089,0.131646,0.121519,0.121519,0.118987,0.103797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [5]:
df_train = df_train.sample(frac=1)

In [6]:
df_test = pd.read_csv("D:/COEP/Project/Main/Final Review/Dataset/mitbih_test.csv", header=None)

Normal

In [7]:
df_train[df_train[187]==0][187]

58281    0.0
41253    0.0
23869    0.0
7631     0.0
39898    0.0
        ... 
24991    0.0
29362    0.0
24239    0.0
57688    0.0
53743    0.0
Name: 187, Length: 72471, dtype: float64

LBBB

In [9]:
df_train[df_train[187]==1][187]

72750    1.0
73808    1.0
74326    1.0
73780    1.0
74660    1.0
        ... 
74376    1.0
73053    1.0
74448    1.0
74341    1.0
73426    1.0
Name: 187, Length: 2223, dtype: float64

RBBB

In [10]:
df_train[df_train[187]==2][187]

76286    2.0
80354    2.0
79181    2.0
76794    2.0
80223    2.0
        ... 
78558    2.0
74931    2.0
75986    2.0
77280    2.0
77435    2.0
Name: 187, Length: 5788, dtype: float64

APC

In [11]:
df_train[df_train[187]==3][187]

80864    3.0
81052    3.0
81057    3.0
80779    3.0
80645    3.0
        ... 
80825    3.0
80798    3.0
80672    3.0
80962    3.0
80932    3.0
Name: 187, Length: 641, dtype: float64

PVC

In [12]:
df_train[df_train[187]==4][187]

82130    4.0
85090    4.0
85496    4.0
86421    4.0
87348    4.0
        ... 
84780    4.0
82254    4.0
82451    4.0
81150    4.0
86382    4.0
Name: 187, Length: 6431, dtype: float64

In [13]:
df_train[187].values

array([0., 0., 0., ..., 0., 0., 0.])

In [14]:
X_train = np.array(df_train[list(range(187))].values)[..., np.newaxis]

In [15]:
Y_train = np.array(df_train[187].values).astype(np.int8)

In [16]:
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [17]:
Y_test = np.array(df_test[187].values).astype(np.int8)

In [18]:
 nclass = 5

In [60]:
input_im = Input(shape=(187, 1))

In [110]:
im = Convolution1D(128, kernel_size=5, activation=activations.relu, padding="valid")(input_im)

In [111]:
im = BatchNormalization()(im)

In [112]:
im = Convolution1D(128, kernel_size=5, activation=activations.relu, padding="valid")(im)

In [113]:
im = BatchNormalization()(im)

In [114]:
im = MaxPool1D(pool_size=2)(im)

In [115]:
im = Dropout(rate=0.1)(im)

In [116]:
im = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(im)

In [117]:
im = BatchNormalization()(im)

In [118]:
im = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(im)

In [119]:
im = BatchNormalization()(im)

In [120]:
im = MaxPool1D(pool_size=2)(im)

In [121]:
im = Dropout(rate=0.1)(im)

In [122]:
im = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(im)

In [123]:
im = BatchNormalization()(im)

In [124]:
im = Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid")(im)

In [125]:
im = BatchNormalization()(im)

In [126]:
im = MaxPool1D(pool_size=2)(im)

In [127]:
im = Dropout(rate=0.1)(im)

In [128]:
im = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(im)

In [129]:
im = BatchNormalization()(im)

In [130]:
im = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(im)

In [131]:
im = BatchNormalization()(im)

In [132]:
im = GlobalMaxPool1D()(im)

In [133]:
im = Dropout(rate=0.2)(im)

In [134]:
d1 = Dense(128, activation=activations.relu, name="d1")(im)

In [135]:
d1 = BatchNormalization()(d1)

In [136]:
d1 = Dropout(rate=0.5)(d1)

In [137]:
d1 = Dense(64, activation=activations.relu, name="d2")(d1)

In [138]:
d1 = Dense(nclass, activation=activations.softmax, name="d3_arr")(d1)

In [139]:
model = models.Model(inputs=input_im, outputs=d1)

In [140]:
opt = optimizers.Adam(0.001)

In [141]:
model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])

In [142]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 187, 1)]          0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 183, 128)          768       
_________________________________________________________________
batch_normalization_18 (Batc (None, 183, 128)          512       
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 179, 128)          82048     
_________________________________________________________________
batch_normalization_19 (Batc (None, 179, 128)          512       
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 89, 128)           0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 89, 128)           0   

In [143]:
chk_pnt_filepath = "Beat_CLassification_Model.h5"

ModelCheckpoint : Saves a model or weights in a file do that the model or weights can be loaded at a later point in time and training can be continued.

https://keras.io/api/callbacks/model_checkpoint/

In [144]:
chk_pnt = ModelCheckpoint(chk_pnt_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

Early Stopping: Stops training when a monitored metric has stopped improving over a specified number of epochs

https://www.ironmanjohn.com/home/i-love-keras-earlystopping

https://keras.io/api/callbacks/early_stopping/

https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

In [145]:
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)

Reduces learning rate by a factor when there is no improvement in the specified metric over the specified number of epochs.

In [146]:
rdlp = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)

In [147]:
callbacks_list = [chk_pnt, early, rdlp]  

In [148]:
model.fit(X_train, Y_train, epochs=100, verbose=2, callbacks=callbacks_list, validation_split=0.1)

Epoch 1/100
2463/2463 - 344s - loss: 0.3017 - acc: 0.9136 - val_loss: 0.1783 - val_acc: 0.9512

Epoch 00001: val_acc improved from -inf to 0.95123, saving model to Beat_CLassification_Model.h5
Epoch 2/100
2463/2463 - 339s - loss: 0.1644 - acc: 0.9557 - val_loss: 0.2901 - val_acc: 0.8971

Epoch 00002: val_acc did not improve from 0.95123
Epoch 3/100
2463/2463 - 339s - loss: 0.1313 - acc: 0.9649 - val_loss: 0.1075 - val_acc: 0.9690

Epoch 00003: val_acc improved from 0.95123 to 0.96905, saving model to Beat_CLassification_Model.h5
Epoch 4/100
2463/2463 - 355s - loss: 0.1121 - acc: 0.9696 - val_loss: 0.1058 - val_acc: 0.9703

Epoch 00004: val_acc improved from 0.96905 to 0.97031, saving model to Beat_CLassification_Model.h5
Epoch 5/100
2463/2463 - 345s - loss: 0.0954 - acc: 0.9735 - val_loss: 0.0691 - val_acc: 0.9794

Epoch 00005: val_acc improved from 0.97031 to 0.97944, saving model to Beat_CLassification_Model.h5
Epoch 6/100
2463/2463 - 339s - loss: 0.0888 - acc: 0.9755 - val_loss: 0.0

In [149]:
model.load_weights(chk_pnt_filepath)

In [150]:
pred_test = model.predict(X_test)
#print(pred_test)
pred_test = np.argmax(pred_test, axis=-1)

In [151]:
f1 = f1_score(Y_test, pred_test, average="macro")

In [152]:
print("Test f1 score : %s "% f1)

Test f1 score : 0.9255069882687508 


In [153]:
acc = accuracy_score(Y_test, pred_test)

In [154]:
print("Test accuracy score : %s "% acc)

Test accuracy score : 0.98780376393203 


In [155]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(Y_test,pred_test))


[[18057    43    11     4     3]
 [   90   457     9     0     0]
 [   33     4  1393    17     1]
 [   21     2    14   125     0]
 [    9     0     6     0  1593]]


In [156]:
print(classification_report(Y_test,pred_test))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     18118
           1       0.90      0.82      0.86       556
           2       0.97      0.96      0.97      1448
           3       0.86      0.77      0.81       162
           4       1.00      0.99      0.99      1608

    accuracy                           0.99     21892
   macro avg       0.94      0.91      0.93     21892
weighted avg       0.99      0.99      0.99     21892

